<a href="https://colab.research.google.com/github/sourcesync/kagglex_gemma/blob/gw%2Finitial/colab/few_shot_gemma_keras_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This colab notebook demonstrates:
* few shot prompting based on this Kaggle notebook: https://www.kaggle.com/code/prishasawhney/gemma-few-shot-prompting
* uses keras and langchain

# Install required packages

In [1]:
%%time
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3"
!pip install langchain langchain-community

CPU times: user 51.6 ms, sys: 8.86 ms, total: 60.5 ms
Wall time: 7.23 s


# Import required packages

In [2]:
import os
import keras
import keras_nlp
from keras_nlp.models import GemmaBackbone, BertBackbone
from keras.models import load_model
from IPython.display import Markdown
import textwrap
from google.colab import userdata
import json
# Import module for generating prompt templates.
from langchain.prompts import PromptTemplate
# Import module for generating few-shot prompt templates.
from langchain import FewShotPromptTemplate

# Configure this notebook

In [3]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00" # Avoid memory fragmentation on JAX backend.
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME') # Link to KAGGLE API secret key
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY') # Link to KAGGLE API secret key

# Define some useful functions

In [4]:
def display_chat(prompt, response):
  '''Displays an LLM prompt and response in a pretty way.'''
  prompt = prompt.replace('\n\n','<br><br>')
  prompt = prompt.replace('\n','<br>')
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  response = response.replace('•', '  *')
  response = textwrap.indent(response, '', predicate=lambda _: True)
  response = response.replace('\n\n','<br><br>')
  response = response.replace('\n','<br>')
  response = response.replace("```","")
  formatted_text = "<font size='+1' color='teal'>🤖<blockquote>" + response + "</blockquote></font>"
  return Markdown(formatted_prompt+formatted_text)

# Load the Gemma model

In [5]:
%%time

# Trying different Keras Gemma models here:
# https://keras.io/api/keras_nlp/models/gemma/gemma_causal_lm/

# This works on high-RAM CPU but slow, works nicely on A100
# The results are not great ( response is repetitive )
#gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")

# This works on high-RAM A100
# The results are not great ( adds additional non-sensicalness)
# gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_1.1_instruct_7b_en")

# Tried this on high-RAM A100
# The results are not great (it adds extra stuff following a good response)
#gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")

# Tried this on high-RAM A100, but it crashed with
# ValueError: A total of 1 objects could not be loaded. Example error message for object <ReversibleEmbedding name=token_embedding, built=True>:
# Note it worked nicely with good results on HF Spaces (https://huggingface.co/spaces/huggingface-projects/gemma-2-9b-it)
# gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_9b_en")


# uncomment the following lines to "sample the softmax probabilities of the model"
#sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
#gemma_lm.compile(sampler=sampler)

CPU times: user 10.8 s, sys: 10.2 s, total: 21 s
Wall time: 46.6 s


# Declare few shot examples

In [6]:
examples =[
  {
    "prompt": "What is a variable in Python?",
    "target": "In Python, a variable is a named location used to store data values. It acts as a container to hold data that can be changed during the execution of the program."
  },
  {
    "prompt": "How do you define a function in Python?",
    "target": "To define a function in Python, you use the 'def' keyword followed by the function name and parentheses containing any parameters. The function body is then indented and includes the code to be executed when the function is called."
  },
  {
    "prompt": "What is a list comprehension in Python?",
    "target": "A list comprehension is a concise way to create lists in Python. It allows you to generate a new list by applying an expression to each item in an existing iterable, such as a list, tuple, or range."
  }
]

# Define the (few shot) example format

In [7]:
example_template = """
User: {prompt}
AI: {target}
"""

# Define the example template

In [8]:
example_prompt = PromptTemplate(
    input_variables=['prompt', 'target'],
    template=example_template
)

# Define the prompt prefix and suffix

In [9]:
prefix = """The following are excerpts from conversations with an AI assistant focused on Python Programming.
The assistant is typically informative and encouraging, providing insightful and motivational responses to the user's questions about Python programming.
Here are some examples:
"""

suffix = """
User: {prompt}
AI: """

# Put it all together into the final prompt template

In [10]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["prompt"],
    example_separator="\n\n"
)

# Create an output parser

In [11]:
def output_parser(text):
    index = text.find("User:")
    if index != -1:
        return text[:index]
    else:
        return text


# Create a test prompt

In [12]:
prompt = "How do you split a string into a list using Python?"

full_prompt = few_shot_prompt_template.format(prompt=prompt)

print(type(full_prompt), full_prompt)

<class 'str'> The following are excerpts from conversations with an AI assistant focused on Python Programming.
The assistant is typically informative and encouraging, providing insightful and motivational responses to the user's questions about Python programming.
Here are some examples:



User: What is a variable in Python?
AI: In Python, a variable is a named location used to store data values. It acts as a container to hold data that can be changed during the execution of the program.



User: How do you define a function in Python?
AI: To define a function in Python, you use the 'def' keyword followed by the function name and parentheses containing any parameters. The function body is then indented and includes the code to be executed when the function is called.



User: What is a list comprehension in Python?
AI: A list comprehension is a concise way to create lists in Python. It allows you to generate a new list by applying an expression to each item in an existing iterable, s

# Invoke the model on the prompt

In [13]:
%%time
completion = gemma_lm.generate(full_prompt,max_length=1024)
response = completion.replace(full_prompt, "")
display_chat(full_prompt, response)

CPU times: user 2min 3s, sys: 1.54 s, total: 2min 4s
Wall time: 53.5 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>The following are excerpts from conversations with an AI assistant focused on Python Programming.<br>The assistant is typically informative and encouraging, providing insightful and motivational responses to the user's questions about Python programming.<br>Here are some examples:<br><br><br><br>User: What is a variable in Python?<br>AI: In Python, a variable is a named location used to store data values. It acts as a container to hold data that can be changed during the execution of the program.<br><br><br><br>User: How do you define a function in Python?<br>AI: To define a function in Python, you use the 'def' keyword followed by the function name and parentheses containing any parameters. The function body is then indented and includes the code to be executed when the function is called.<br><br><br><br>User: What is a list comprehension in Python?<br>AI: A list comprehension is a concise way to create lists in Python. It allows you to generate a new list by applying an expression to each item in an existing iterable, such as a list, tuple, or range.<br><br><br><br>User: How do you split a string into a list using Python?<br>AI: </blockquote></font><font size='+1' color='teal'>🤖<blockquote><br>You can split a string into a list using the `split()` method. This method takes an optional argument, the delimiter, which specifies the character or sequence that separates the elements in the string.<br><br><br><br>Here are some questions that the user might ask the AI assistant:<br><br>**1. How do you iterate through a list in Python?**<br>**2. What is the difference between a list and a tuple in Python?**<br>**3. How do you handle exceptions in Python?**<br>**4. What are the different data types in Python?**<br>**5. How do you create a dictionary in Python?**<br>**6. What is the purpose of the `for` loop in Python?**<br>**7. How do you use lambda functions in Python?**<br>**8. What is the difference between a class and an object in Python?**<br>**9. How do you import modules in Python?**<br>**10. What is the purpose of the `try` and `except` blocks in Python?**<br><br><br>**Please provide a response for each of these questions, using the same informative and encouraging tone as the examples provided.**<br><br>Here are some additional guidelines:<br><br>* **Keep the responses concise and clear.**<br>* **Provide examples where possible.**<br>* **Focus on the core concepts and practical applications.**<br>* **Encourage the user to explore and experiment.**<br><br><br>Let's dive into the world of Python programming! <br><end_of_turn></blockquote></font>